In [5]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import argrelextrema

In [6]:
def get_stock_data(ticker,start_date='2014-01-29', interval='1d', period='2y'):
    """
    Fetch historical stock data.
        ticker: name of the fund
        interval: time interval for fetching data ('1d', '1h', '1m', etc.)
        period: period to fetch data for ('1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max')
    """
    stock = yf.Ticker(ticker)
    data = stock.history(start=start_date, interval=interval, period=period)
    if not isinstance(data, pd.DataFrame) or data.empty:
        raise ValueError(f"No valid data found for ticker {ticker}.")
    return data


In [7]:
def find_extrema(data, column='Close', order=5):
    """
    Find local and global extrema of stock prices.
    """
    prices = data[column]
    
    # Local minima and maxima using argrelextrema
    local_maxima = argrelextrema(prices.values, np.greater, order=order)[0]
    local_minima = argrelextrema(prices.values, np.less, order=order)[0]

    # Global extrema (max and min)
    global_max = prices.idxmax()  # Date of global maximum
    global_min = prices.idxmin()  # Date of global minimum

    # Return results with corresponding dates and values
    return {
        'local_maxima': data.iloc[local_maxima],
        'local_minima': data.iloc[local_minima],
        'global_max': (global_max, prices[global_max]),
        'global_min': (global_min, prices[global_min])
    }

In [8]:
import os


def analyze_extract_stocks(tickers, interval='1d', period='5y', order=5,start_date='2012-06-30'):
    extrema_data = []
    for ticker in tickers:
        print(f"Fetching data for {ticker}...")
        try:
            data = get_stock_data(ticker, interval=interval, period=period, start_date=start_date)
            data.reset_index(inplace=True)  # reset index to access 'Date' as a column
            print(f"Data for {ticker} fetched successfully. Columns: {data.columns.tolist()}")
            
            data = data[['Date', 'Close']]
            data.columns = ['date', 'price']
            
            extrema = find_extrema(data, column='price', order=order)
            extrema_data.extend([{
                'date': row['date'], 'type': 'local_minima', 'ticker': ticker, 'price': row['price']
            } for _, row in extrema['local_minima'].iterrows()])
            extrema_data.extend([{
                'date': row['date'], 'type': 'local_maxima', 'ticker': ticker, 'price': row['price']
            } for _, row in extrema['local_maxima'].iterrows()])

            global_max = data.loc[data['price'].idxmax()]
            global_min = data.loc[data['price'].idxmin()]
            extrema_data.append({
                'date': global_max['date'], 'type': 'global_max', 'ticker': ticker, 'price': global_max['price']
            })
            extrema_data.append({
                'date': global_min['date'], 'type': 'global_min', 'ticker': ticker, 'price': global_min['price']
            })

        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    # Convert to DataFrame and save to CSV
    if extrema_data:
        extrema_df = pd.DataFrame(extrema_data)
        base_filename = 'extrema_dates'
        extension = '.csv'
        filename = base_filename + extension
        counter = 1
        
        while os.path.exists(filename):
            filename = f"{base_filename}{counter}{extension}"
            counter += 1
        
        extrema_df.to_csv(filename, index=False)
        print(f"Extrema data saved to '{filename}'.")
    else:
        print("No extrema data to save.")

In [12]:
tickers = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'NVDA', 'AMD', 'AMZN', 'NFLX', 'INTC', 'XOM', 'CVX', 'OXY', 'MRNA', 'PFE', 'BIIB', 'GME', 'AMC', 'BB', 'PLUG', 'PLTR', 'ROKU', 'ZM', 'SQ', 'PYPL', 'UBER', 'LYFT', 'T', 'VZ', 'TMUS', 'S', 'TM', 'F', 'GM', 'NIO', 'LI', 'XPEV', 'PBM', 'SRM', 'NTRP', 'ENTO', 'CRKN', 'JZXN', 'ABP']
ticker3=['PYPL']
analyze_extract_stocks(ticker3,interval='1d', period='5y', order=5,start_date='2015-06-01')



Fetching data for PYPL...
Data for PYPL fetched successfully. Columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
Extrema data saved to 'extrema_dates3.csv'.


In [10]:
ticker2=['^IXIC']
analyze_extract_stocks(ticker2)

Fetching data for ^IXIC...
Data for ^IXIC fetched successfully. Columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
Extrema data saved to 'extrema_dates4.csv'.
